In [1]:
# Imports
from distutils.command.config import config
import pathlib
import torch

# import albumentations
import numpy as np
import os
import glob

from monai.data import ArrayDataset, CacheDataset
from monai.transforms import (
    Activations,
    EnsureChannelFirst,
    AsDiscrete,
    Compose,
    LoadImage,
    RandSpatialCrop,
    Resize,
    ScaleIntensity,
)

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from skimage.transform import resize
# from customdatasets import SegmentationDataSet1
from transformation import (
    ComposeDouble,
    AlbuSeg3d,
    FunctionWrapperDouble,
    normalize_01,
    create_dense_target,
)
from unet import UNet
from trainer import Trainer
from dataset import AtrophyDataset


TypeError: an integer is required (got type bytes)

In [2]:
# root directory
# root = pathlib.Path.cwd() / "Carvana"
root = "/Users/linhle/Documents/UC_Davis/Fan_Lab/atrophyData/"


def get_filenames_of_path(path: pathlib.Path, ext: str = "*"):
    """Returns a list of files in a directory/path. Uses pathlib."""
    filenames = [file for file in path.glob(ext) if file.is_file()]
    return filenames

inputs = sorted(glob.glob(os.path.join(root,"trainData/NIFTI/", "*3DT1_Warped.nii")))
targets = sorted(glob.glob(os.path.join(root,"NewJacs/NIFTI/", "*MDT3_mask.nii")))


# input and target files
# inputs = get_filenames_of_path(root / "trainData")
# targets = get_filenames_of_path(root / "NewJacs")

print(targets[:10])

['/Users/linhle/Documents/UC_Davis/Fan_Lab/atrophyData/NewJacs/NIFTI/10100_042903_091316_2.0_ONLM_Jac_Warped_Annualized_MDT3_mask.nii', '/Users/linhle/Documents/UC_Davis/Fan_Lab/atrophyData/NewJacs/NIFTI/2353_112712_111513_2.0_ONLM_Jac_Warped_Annualized_MDT3_mask.nii', '/Users/linhle/Documents/UC_Davis/Fan_Lab/atrophyData/NewJacs/NIFTI/2369_100709_112613_2.0_ONLM_Jac_Warped_Annualized_MDT3_mask.nii', '/Users/linhle/Documents/UC_Davis/Fan_Lab/atrophyData/NewJacs/NIFTI/2448_072407_112013_2.0_ONLM_Jac_Warped_Annualized_MDT3_mask.nii', '/Users/linhle/Documents/UC_Davis/Fan_Lab/atrophyData/NewJacs/NIFTI/2524_021804_082516_2.0_ONLM_Jac_Warped_Annualized_MDT3_mask.nii', '/Users/linhle/Documents/UC_Davis/Fan_Lab/atrophyData/NewJacs/NIFTI/2580_051904_051316_2.0_ONLM_Jac_Warped_Annualized_MDT3_mask.nii', '/Users/linhle/Documents/UC_Davis/Fan_Lab/atrophyData/NewJacs/NIFTI/2771_120909_093016_2.0_ONLM_Jac_Warped_Annualized_MDT3_mask.nii', '/Users/linhle/Documents/UC_Davis/Fan_Lab/atrophyData/NewJac

In [3]:
# training transformations and augmentations
'''
transforms_training = ComposeDouble(
    [
        FunctionWrapperDouble(
            resize, input=True, target=False, output_shape=(128, 128, 3)
        ),
        FunctionWrapperDouble(
            resize,
            input=False,
            target=True,
            output_shape=(128, 128),
            order=0,
            anti_aliasing=False,
            preserve_range=True,
        ),
        AlbuSeg2d(albumentations.HorizontalFlip(p=0.5)),
        FunctionWrapperDouble(create_dense_target, input=False, target=True),
        FunctionWrapperDouble(
            np.moveaxis, input=True, target=False, source=-1, destination=0
        ),
        FunctionWrapperDouble(normalize_01),
    ]
)

# validation transformations
transforms_validation = ComposeDouble(
    [
        FunctionWrapperDouble(
            resize, input=True, target=False, output_shape=(128, 128, 3)
        ),
        FunctionWrapperDouble(
            resize,
            input=False,
            target=True,
            output_shape=(128, 128),
            order=0,
            anti_aliasing=False,
            preserve_range=True,
        ),
        FunctionWrapperDouble(create_dense_target, input=False, target=True),
        FunctionWrapperDouble(
            np.moveaxis, input=True, target=False, source=-1, destination=0
        ),
        FunctionWrapperDouble(normalize_01),
    ]
)
'''
# transforms_training = Compose(
#     [
#         LoadImage(image_only=True),
#         ScaleIntensity(),
#         EnsureChannelFirst(),
#         # RandSpatialCrop((128, 96, 128), random_size=False),
#     ]
# )
# transforms_target = Compose(
#     [
#         LoadImage(image_only=True),
#         EnsureChannelFirst(),
#         # RandSpatialCrop((128, 96, 128), random_size=False),
#     ]
# )

# training transformations and augmentations
transforms_training = ComposeDouble([
    # AlbuSeg3d(albumentations.HorizontalFlip(p=0.5)),
    FunctionWrapperDouble(create_dense_target, input=False, target=True),
    FunctionWrapperDouble(np.moveaxis, input=True, target=False, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01)
])

# validation transformations
transforms_validation = ComposeDouble([
    FunctionWrapperDouble(create_dense_target, input=False, target=True),
    FunctionWrapperDouble(np.moveaxis, input=True, target=False, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01)
])

pre_transforms = ComposeDouble([
    FunctionWrapperDouble(resize,
                          input=True,
                          target=False,
                          output_shape=(128, 96, 128)),
    FunctionWrapperDouble(resize,
                          input=False,
                          target=True,
                          output_shape=(128, 96, 128),
                        #   order=0,
                          anti_aliasing=False,
                          preserve_range=True),
])

In [4]:

# random seed
random_seed = 42

# split dataset into training set and validation set
train_size = 0.8  # 80:20 split

inputs_train, inputs_valid = train_test_split(
    inputs, random_state=random_seed, train_size=train_size, shuffle=True
)

targets_train, targets_valid = train_test_split(
    targets, random_state=random_seed, train_size=train_size, shuffle=True
)

# inputs_train, inputs_valid = inputs[:80], inputs[80:]
# targets_train, targets_valid = targets[:80], targets[:80]

# dataset training
# dataset_train = ArrayDataset(
#     inputs_train, transforms_training, targets_train, transforms_target
# )


dataset_train = AtrophyDataset(inputs=inputs_train,
                                targets=targets_train,
                                transform=transforms_training,
                                use_cache=True,
                                pre_transform=pre_transforms)

# dataset validation
# dataset_val = ArrayDataset(
#     inputs_valid, transforms_training, targets_valid, transforms_training
# )

dataset_val = AtrophyDataset(inputs=inputs_valid,
                                targets=targets_valid,
                                transform=transforms_validation,
                                use_cache=True,
                                pre_transform=pre_transforms)

# dataloader training
dataloader_training = DataLoader(dataset=dataset_train, batch_size=8, shuffle=True)

# dataloader validation
dataloader_validation = DataLoader(dataset=dataset_val, batch_size=2, shuffle=True)

Caching: 100%|██████████| 19/19 [00:02<00:00,  7.08it/s]


In [5]:
for i,j in dataloader_training:
    print("image shape ", {i.shape})
    print("label shape ", {j.shape})
    break

KeyboardInterrupt: 

In [ ]:
# create DatasetViewer instances
from visual import DatasetViewer

dataset_viewer_training = DatasetViewer(dataset_train)
dataset_viewer_validation = DatasetViewer(dataset_val)

In [ ]:
# open napari instance for training dataset
# navigate with 'n' for next and 'b' for back on the keyboard
dataset_viewer_training.napari()

AttributeError: 'str' object has no attribute 'name'

In [ ]:
# open napari instance for validation dataset
# navigate with 'n' for next and 'b' for back on the keyboard
dataset_viewer_validation.napari()

In [1]:
# device (using GPU)
device = torch.device("mps") 
# if torch.cuda.is_available() else torch.device("cpu")

# model
model = UNet(
    in_channels=1,
    out_channels=1,
    n_blocks=5,
    start_filters=32,
    activation="relu",
    normalization="batch",
    conv_mode="same",
    dim=3,
).to(device)

# criterion
criterion = torch.nn.MSELoss()

# optimizer
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
lr = 1e-2 # learning rate 
optimizer = torch.optim.Adam(model.parameters(), lr)


NameError: name 'torch' is not defined

In [ ]:
# trainer
trainer = Trainer(
    model=model,
    device=device,
    criterion=criterion,
    optimizer=optimizer,
    training_DataLoader=dataloader_training,
    validation_DataLoader=dataloader_validation,
    lr_scheduler=None,
    epochs=2,
    epoch=0,
    notebook=True,
)

# start training
training_losses, validation_losses, lr_rates = trainer.run_trainer()

Progress:   0%|          | 0/2 [00:00<?, ?it/s]

Training:   0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: Given groups=1, weight of size [32, 1, 3, 3, 3], expected input[1, 8, 128, 128, 4] to have 1 channels, but got 8 channels instead

In [ ]:
# save the model
model_name = "atrophy_model.pt"
torch.save(model.state_dict(), root, model_name)

In [ ]:
# device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# model
model = UNet(
    in_channels=3,
    out_channels=2,
    n_blocks=4,
    start_filters=32,
    activation="relu",
    normalization="batch",
    conv_mode="same",
    dim=2,
).to(device)

# criterion
criterion = torch.nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
from lr_rate_finder import LearningRateFinder

lrf = LearningRateFinder(model, criterion, optimizer, device)
lrf.fit(dataloader_training, steps=1000)

In [ ]:
lrf.plot()

In [ ]:
from visual import plot_training

fig = plot_training(
    training_losses,
    validation_losses,
    lr_rates,
    gaussian=True,
    sigma=1,
    figsize=(10, 4),
)